In [1]:
from Kaggle_Titanic_utls import *
import warnings
warnings.filterwarnings('ignore')

In [2]:
path = Path('.')

In [3]:
# Import data
data = pd.read_csv(path / "train.csv")

In [4]:
data.shape

(891, 12)

In [5]:
data.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
data['Survived'].sum()/len(data)

0.3838383838383838

In [8]:
#Fare funzione Alessandro

The classes are sufficiently balanced - it makes sense to consider accuracy as performance metrics.

Embarked only has 2 missing values out of 891 - it makes sense to get rid of them altogether.

In [9]:
data.dropna(subset=['Embarked'], inplace=True)

In [10]:
labels = data[['Survived']]

data.drop(['Cabin', 'Ticket', 'PassengerId', 'Survived'], axis=1, inplace=True)

In [11]:
data.corr().style.background_gradient(cmap ='cool')\
        .set_properties(**{'font-size': '15px'}) 

,Pclass,Age,SibSp,Parch,Fare
Pclass,1.000000,-0.365902,0.081656,0.016824,-0.548193
Age,-0.365902,1.000000,-0.307351,-0.187896,0.093143
SibSp,0.081656,-0.307351,1.000000,0.414542,0.160887
Parch,0.016824,-0.187896,0.414542,1.000000,0.217532
Fare,-0.548193,0.093143,0.160887,0.217532,1.000000


There are no highly correlated features (maybe do feature extraction?).

We can extract the title of each passenger from the name **(which is most likely not useful in the prediction. We can check this though)**.

In [12]:
title = data.Name.apply(lambda x:x.split(',')[1].split('.')[0].strip())

In [13]:
title.value_counts()

Mr              517
Miss            181
Mrs             124
Master           40
Dr                7
Rev               6
Mlle              2
Col               2
Major             2
Lady              1
Mme               1
Ms                1
Don               1
Sir               1
Capt              1
the Countess      1
Jonkheer          1
Name: Name, dtype: int64

In [14]:
data['Title'] = title
data.drop('Name', axis=1, inplace=True)

In [15]:
labels.shape, data.shape

((889, 1), (889, 8))

In [16]:
data.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Title'], dtype='object')

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

In [19]:
numeric_features = ['Age', 'SibSp', 'Fare', 'Parch']
categorical_features = ['Embarked', 'Sex', 'Pclass', 'Title']

In [34]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = OneHotEncoder(handle_unknown='ignore')
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

pipeline = Pipeline(steps=[('preprocessor', preprocessor), 
                           ('classifier', KNeighborsClassifier())])

In [35]:
from sklearn import set_config

set_config(display='diagram')
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'SibSp', 'Fare',
                                                   'Parch']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Embarked', 'Sex', 'Pclass',
                                                   'Title'])])),
                ('classifier', KNeighborsClassifier())])

In [37]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2,
                                                    random_state=0)


clf.fit(X_train, y_train)
print("model score: %.3f" % pipeline.score(X_test, y_test))

NameError: name 'clf' is not defined

In [17]:
models = {'log': LogisticRegression(), 
          'tree': DecisionTreeClassifier(), 
          'svc': SVC(), 
          'perceptron': Perceptron(), 
          'knn': KNeighborsClassifier()}


for i, j in models.items():
    print(i, base_pip(j, data, labels))

train_tr shape (755, 29)
log (0.833112582781457, 0.8134328358208955)
train_tr shape (755, 29)
tree (0.9827814569536424, 0.7985074626865671)
train_tr shape (755, 29)
svc (0.8423841059602649, 0.8208955223880597)
train_tr shape (755, 29)
perceptron (0.7311258278145696, 0.7611940298507462)
train_tr shape (755, 29)
knn (0.8649006622516556, 0.8582089552238806)


## NOTA: dobbiamo inserire anche i gaussian NB che al momento danno dei problemi perché non vogliono una matrice sparsa come input.

## To visualize the pipeline

from sklearn import set_config

set_config(display='diagram')
clf

It seems that the k-nearest neighbours does the best job among the models considered. Support Vector Classifier and Logistic Regression also score reasonably well (random guessing would produce a $\sim 62\%$ accuracy).

We now will:

1. Evaluate other imputation methods on selected models;
2. Choose the best imputation method;
3. Do a grid search on parameters for the best model(s) and imputation method(s).

## Imputation

In [18]:
selected_models = ['knn', 'log', 'svc'] 

imputation_methods = ['mean', 'median', 'function', 'zeros']

for i in selected_models:
    print(i)
    for j in imputation_methods:
        print('\t',j, base_pip(models[i], data, labels, imputer_s=j))
    print(' ')
        

knn
train_tr shape (755, 29)
	 mean (0.8609271523178808, 0.835820895522388)
train_tr shape (755, 29)
	 median (0.8649006622516556, 0.8582089552238806)
train_tr shape (755, 29)
	 function (0.8609271523178808, 0.8432835820895522)
train_tr shape (755, 29)
	 zeros (0.8582781456953642, 0.8432835820895522)
 
log
train_tr shape (755, 29)
	 mean (0.833112582781457, 0.8134328358208955)
train_tr shape (755, 29)
	 median (0.833112582781457, 0.8134328358208955)
train_tr shape (755, 29)
	 function (0.833112582781457, 0.8134328358208955)
train_tr shape (755, 29)
	 zeros (0.8304635761589404, 0.8134328358208955)
 
svc
train_tr shape (755, 29)
	 mean (0.8423841059602649, 0.8208955223880597)
train_tr shape (755, 29)
	 median (0.8423841059602649, 0.8208955223880597)
train_tr shape (755, 29)
	 function (0.8423841059602649, 0.8208955223880597)
train_tr shape (755, 29)
	 zeros (0.8397350993377484, 0.8208955223880597)
 


Different imputation methods have little impact on train and test for LR and SVC. KNN is slightly less stable from this standpoint, but it remains the best performing model, with the median as imputation method.

## Grid Search

Now that we have chosen our model, we can grid search parameters.

In [19]:
param_grid = {'weights': ['uniform', 'distance'], 
               'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 
               'n_neighbors': np.linspace(1, 15, 15).astype(int)}

best_cv = base_pip(models['knn'], data, labels, 
                   imputer_s='median', option='grid_search', 
                   param_grid=param_grid, cv=3)

train_tr shape with gscv (889, 29)
Fitting 3 folds for each of 120 candidates, totalling 360 fits


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-3)]: Done  40 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-3)]: Done 360 out of 360 | elapsed:    3.3s finished


In [20]:
models = {'best': best_cv['best_estimator']}


for i, j in models.items():
    print(i, base_pip(j, data, labels))

train_tr shape (755, 29)
best (0.8370860927152318, 0.8283582089552238)


## Submission

In [21]:
X_unknown = pd.read_csv(path / 'test.csv')
X_unknown.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [22]:
X_unknown.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [23]:
X_unknown.dropna(subset=['Fare'], inplace=True)

In [24]:
X_unknown.drop(['Cabin', 'Ticket', 'PassengerId'], axis=1, inplace=True)

In [25]:
title_ = X_unknown.Name.apply(lambda x:x.split(',')[1].split('.')[0].strip())

X_unknown['Title'] = title

In [26]:
X_unknown.drop(['Name'], axis=1, inplace=True)

In [27]:
X_unknown.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417 entries, 0 to 417
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    417 non-null    int64  
 1   Sex       417 non-null    object 
 2   Age       331 non-null    float64
 3   SibSp     417 non-null    int64  
 4   Parch     417 non-null    int64  
 5   Fare      417 non-null    float64
 6   Embarked  417 non-null    object 
 7   Title     416 non-null    object 
dtypes: float64(2), int64(3), object(3)
memory usage: 29.3+ KB


In [35]:
pipeline_wparams = best_cv['pipeline'].set_params(**best_cv['pipeline_params'])
X_unknown.dropna(subset=['Title'], inplace=True)
pipeline = best_cv['pipeline']
X_unknown_tr = pipeline.fit_transform(X_unknown)

In [33]:
X_train = pipeline.fit_transform(data)
X_train.shape

(889, 29)

In [32]:
X_unknown_tr = X_train.align(X_unknown_tr, axis=1)
X_unknown_tr

TypeError: unsupported type: <class 'numpy.ndarray'>

In [33]:
predictions = best_cv['best_estimator'].predict(X_unknown_tr)

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 20 while Y.shape[1] == 28